In [ ]:
# !pip install torch==2.1.2 torchvision==0.16.2 torchaudio==2.1.2 --index-url https://download.pytorch.org/whl/cu121

In [ ]:
# !pip install https://github.com/jllllll/bitsandbytes-windows-webui/releases/download/wheels/bitsandbytes-0.41.0-py3-none-win_amd64.whl

In [1]:
# !pip install -q datasets transformers accelerate loralib sentencepiece gradio fire peft wandb peft

In [1]:
import zipfile
import json
import datetime

import torch
from torch import nn
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
from datasets import Dataset
import peft
from peft import PeftModelForSequenceClassification, LoraConfig, get_peft_model
import evaluate
import wandb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
from loguru import logger
import numpy as np

In [2]:
print(torch.cuda.is_available())

True


In [3]:
data_zip = zipfile.ZipFile('D:\My_projects\Grapix\garpix-solution\data\small_boxes.zip', 'r')

In [4]:
with data_zip.open(data_zip.filelist[0].filename) as file:
    json_file = json.loads(file.read())

In [6]:
boxes = pd.json_normalize(json_file['first_visual']['boxes'])[[
    'mass','size.width','size.height', 'size.length'
    ]].to_numpy().flatten()

In [7]:
target = json_file['first_visual']['calculation_info']['density_percent']

In [8]:
def json_to_array(json):
    
    boxes = pd.json_normalize(json['first_visual']['boxes'])[[
        'mass', 'size.width', 'size.height', 'size.length'
        ]]
    target = json['first_visual']['calculation_info']['density_percent']
    
    return boxes.to_numpy().flatten(), target

In [9]:
arr = []
targets = []

for data_info in data_zip.filelist:
    with data_zip.open(data_info.filename) as file:
        json_file = json.loads(file.read())
        try:
            boxes, target = json_to_array(json_file)
            # if len(boxes) <= 512:
            boxes = np.array2string(
                boxes, separator=' ', formatter={'float_kind': lambda x: str(int(x))}
            ).replace('[', '').replace(']', '').replace('\n', '')
            arr.append(str(boxes))
            targets.append(round(target))
            # else:
            #     logger.info(f'{data_info.filename} length is more than 512. Skip.')
            #     pass
        except Exception as e:
            logger.warning(f'\nError {e} \n In file {data_info.filename}')
            pass

In [10]:
len(arr), len(targets)

(4999, 4999)

In [16]:
df = pd.DataFrame()

In [17]:
df['text'] = arr
df['labels'] = targets

In [18]:
len(set(targets))

89

In [20]:
train, val = train_test_split(df, test_size=0.2, random_state=42, shuffle=True)
train_data = Dataset.from_pandas(train)
val_data = Dataset.from_pandas(val)

In [55]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=512)

In [67]:
train_dataset = train_data.map(tokenize_function, batched=True)
val_dataset = val_data.map(tokenize_function, batched=True)

Map:   0%|          | 0/3999 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [57]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", 
                                                           problem_type="multi_label_classification",
                                                           load_in_8bit=True,
                                                           num_labels=len(set(targets)))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [58]:
peft_config = LoraConfig(
    task_type=peft.TaskType.SEQ_CLS,
    target_modules=['query', 'key', 'value'],
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none"
)

In [59]:
model.add_adapter(peft_config)

In [37]:
# peft_former = get_peft_model(model, peft_config)

In [60]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [61]:
print_trainable_parameters(model)

trainable params: 3538944 || all params: 113089625 || trainable%: 3.129326850274727


In [62]:
training_args = TrainingArguments(output_dir="test_trainer",
                                  no_cuda=False,
                                  learning_rate=1e-3,
                                  push_to_hub=False,
                                  num_train_epochs=1,
                                  evaluation_strategy='steps',
                                  eval_steps=50,
                                  logging_strategy="steps",
                                  logging_steps=50
                                  )

metric = evaluate.load("accuracy")

In [63]:
def compute_metrics(eval_pred):

    logits, labels = eval_pred

    predictions = np.argmax(logits, axis=-1)

    return metric.compute(predictions=predictions, references=labels)

In [64]:
"cuda:0" if torch.cuda.is_available() else "cpu"

'cuda:0'

In [65]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

In [66]:
trainer.train()

  0%|          | 0/500 [00:00<?, ?it/s]

ValueError: Target size (torch.Size([8])) must be the same as input size (torch.Size([8, 89]))